In [11]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.graph_objs import *
# from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True) #use plotly offline

#file setup
sdir='' #use to set a directory if this script is not in the same folder as the excel
filename='Polarities.xlsx' #excel file name
sheet = "Ratings" #tab name
dfrt = pd.read_excel(sdir+filename, sheet_name=sheet)
sheet = "Overlays" #tab name
dfoy = pd.read_excel(sdir+filename, sheet_name=sheet)
sheet = "Settings" #tab name
dfset = pd.read_excel(sdir+filename, sheet_name=sheet)

#quick check to make sure things are copacetic
display(dfrt)
# display(dfrn)
display(dfoy)
display(dfset)

attrs = dfrt.columns.values[1:-1] #grab the attribute names
#rename the first and last columns for easier access
dfrt=dfrt.rename(columns = {list(dfrt)[0]:'idea'}) 
dfrt=dfrt.rename(columns = {list(dfrt)[-1]:'notes'})

display(dfrt)
display(attrs)

rating_range=10


,Idea/Example,Typography,Animation,Reach,Feedback,Polish,Notes
0,Medium,3,2,10,8,10,Animation available through gif<br /> or embed...
1,Powerpoint,5,6,3,3,5,NaN
2,Keynote,10,10,3,3,10,NaN
3,Personal Blog,8,10,5,9,8,NaN


,Rows,Unnamed: 1,Unnamed: 2,Unnamed: 3
0,2,3,4.0,5.0
1,4,3,NaN,NaN
2,2,5,NaN,NaN


,use_dual_labels
0,no


,idea,Typography,Animation,Reach,Feedback,Polish,notes
0,Medium,3,2,10,8,10,Animation available through gif<br /> or embed...
1,Powerpoint,5,6,3,3,5,NaN
2,Keynote,10,10,3,3,10,NaN
3,Personal Blog,8,10,5,9,8,NaN


array(['Typography', 'Animation', 'Reach', 'Feedback', 'Polish'], dtype=object)

In [12]:
import math 
for index, row in dfrt.iterrows():    # for each of the ratings rows
    d = []
    
    idea= row['idea'] # idea name
    notes= row['notes'] # notes content
    upper_range=attrs[index] 
    
    stats = row.drop('idea').drop('notes') # get just the numbers
    d.append(Scatterpolar(
      r = stats, # distance from center
      theta = attrs, # angle, which is categorical
      fill = 'toself',
      hoverinfo='r', #display the number on hover
      marker = dict(
        color = "#8090c7",
        symbol = "square",
        size = 8
      )
    ))

    annot=[]

    if (str(notes) != 'nan'): #if notes are included, add them to annotations
        annot.append(
            dict(
                text="Notes: " + notes,
                x=-.1,
                y=0.3,
                showarrow=False,
                align='left'
            ))
     
    # add an annotation with the ratings
    ratings = ""
    ratingvals = "<br>"
    if use_dual_labels:
        for index, row in dfrn.iterrows():
            ratings += dfrn.iloc[index]["Left"]+"/"
            ratings +=dfrn.iloc[index]["Right"]+":  " 
            ratings+= "<br>"
            ratingvals += str(stats[dfrn.iloc[index]["Abbv"]]) + "<br>"
    else:
        for index in attrs:
            ratings += str(index) + ": <br />" 
            ratingvals += str(stats[index]) + "<br />"

    # Rating Labels    
    annot.append(
        dict(
            text="<b>Ratings (1/10):</b> <br>" + ratings,
            x=-.1,
            y=1,
            showarrow=False,
            align='left'
        ))
    # Rating Values, stacked neatly
    annot.append(
        dict(
            text=ratingvals,
            x=.07,
            y=.95507,
            showarrow=False,
            align='left'
        ))
    
    #set up the graph layout
    layout = go.Layout(
      polar = dict(
        radialaxis = dict(
          visible = True,
          range = [0, rating_range], #range is hardcoded from 1-10
          tickangle=0,
        ),
        angularaxis = dict(
          visible = True,
        ),
      ),
      showlegend = False,
      annotations=annot,
      title=idea
    )

    fig = Figure(data=d, layout=layout)
    iplot(fig)

In [98]:
# create overlays of the ones you'd like to compare. 
for index, row in dfoy.iterrows():
#     display(row)
    d = []
    
    row = row[~pd.isnull(row)] # drop any NANs that arise from uneven row length
    title = ''
    for ind, idd in enumerate(row):
#         display(idd)
        row_stats = dfrt.iloc[int(idd-2)]
#         display(row_stats)
        idea= row_stats['idea']
        title += " vs " + idea if ind > 0 else idea
        stats = row_stats.drop('idea').drop('notes')
        d.append(Scatterpolar(
          r = stats,
          theta = attrs,
          fill = 'toself',
          hoverinfo='r',
          name=idea,
          marker = dict(
            symbol = "square",
            size = 8
          )
        ))

    annot=[]

    layout = go.Layout(
      polar = dict(
        radialaxis = dict(
          visible = True,
          range = [0, rating_range],
          tickangle=0,
#           angle=90,
        ),
        angularaxis = dict(
          visible = True,
        ),
      ),
      showlegend = True,
      annotations=annot,
      title=title
    )

    fig = Figure(data=d, layout=layout)
    iplot(fig)